In [5]:
from sklearn import tree
import pandas as pd
import os

In [6]:
df = pd.read_csv("16_columns_coerced.csv")


In [7]:
df = df.fillna(0)


In [8]:
# Changing award values from 'number of awards won' to 'whether or not it won an award'
df['Awards'].values[df['Awards'] > 0] = 1

df = df.drop(["Unnamed: 0"], axis=1)

# Removing some percentage of non-Oscar winners
df = df.drop(df.query('Awards == 0').sample(frac=.4).index)

# Define the target as whether or not a movie won an award
target = df["Awards"]

In [9]:
data = df.drop("Awards", axis=1)
feature_names = data.columns
data.head()

,IMDB,Metacritic,Rotten Tomatoes,Rank,Worldwide Box Office,Actors_1,Actors_2,Actors_3,directors_1,directors_2,Country_1,language_1,language_2,Genre_1,Genre_2
0,8.9,94,0.94,726.0,212928762,5771,232,3537,1352,0,63,27,92,5,7
1,6.6,53,0.52,107.0,708996336,293,1811,2535,1041,0,63,27,0,0,1
2,9.3,80,0.91,3935.0,28307092,5770,4343,668,511,0,63,27,0,7,0
3,8.5,87,0.96,301.0,385524784,4175,1095,3561,1457,0,63,27,0,1,4
5,7.8,60,0.70,2338.0,61466436,5329,3095,2663,1380,0,63,27,92,1,4


In [10]:
# Split the data using train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [11]:
# Create a Decision Tree Classifier
clf = tree.DecisionTreeClassifier()

In [12]:
# Fit the classifier to the data
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
# Calculate the accuracy for the test data
clf.score(X_test, y_test)

0.867109634551495

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))


0.9269102990033222


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [15]:
#RANDOM FOREST
labled_weights = pd.DataFrame(sorted(zip(rf.feature_importances_, feature_names), reverse=True))
labled_weights = labled_weights.rename(columns={0:"Weight", 1: "Name"})
labled_weights = labled_weights.set_index('Name')
labled_weights['Weight'] = labled_weights['Weight'] * 100
labled_weights['Weight'] = labled_weights['Weight'].map("{:,.2f}%".format)
labled_weights


,Weight
Name,
IMDB,15.38%
Rotten Tomatoes,12.96%
Metacritic,12.71%
Rank,11.12%
Worldwide Box Office,8.77%
Actors_2,7.02%
directors_1,6.83%
Actors_3,5.37%
Genre_2,5.22%


In [16]:
#DECISION TREE
from sklearn.metrics import confusion_matrix
average_precision = confusion_matrix(y_test, clf.predict(X_test))

random_forest_matrix = pd.DataFrame(average_precision)
random_forest_matrix.rename(columns={0:"Predicted NO", 1: "Predicted YES"})
actuals = ['Actual YES','Actual NO']
random_forest_matrix[''] = actuals

random_forest_matrix = random_forest_matrix.set_index('')
random_forest_matrix = random_forest_matrix.rename(columns={0:"Predicted NO", 1: "Predicted YES"})

random_forest_matrix

,Predicted NO,Predicted YES
,,
Actual YES,496,47
Actual NO,33,26
